In [1]:
# Author: Chandranil Chakraborttii
# Project: Learning I/O Access Patterns to Improve Prefetching in SSDs
# Paper Link :https://www.researchgate.net/profile/Chandranil_Chakraborttii/publication/344379801_Learning_IO_Access_Patterns_to_Improve_Prefetching_in_SSDs/links/5f6e28fba6fdcc00863adb13/Learning-I-O-Access-Patterns-to-Improve-Prefetching-in-SSDs.pdf
# Loading libraries


from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from functools import reduce
import pickle
import tarfile
import numpy as np
import re
import os
import time

In [2]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb


from tensorflow.python.client import device_lib
import numpy as np
import csv
import pandas as pd
import sys
import os
import glob
import tensorflow as tf


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"



In [4]:
# Load all libraries 
# Load data (trace 1) 
# Map, order by frequency 
# Train model 
# Load another data (trace 2)
# Map, order by frequency - Same as trace 1
# Load model
# Use the model to predict on the new trace
# Note and compare results


import numpy as np
import pandas as pd
import glob
import csv
import sys
import os
import math
import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
from sklearn import metrics
from keras.layers.core import Dense, Activation
from keras.callbacks import ModelCheckpoint
from numpy import insert
from sklearn import preprocessing
from collections import Counter
from numpy import array
from numpy import hstack
from keras.preprocessing.sequence import TimeseriesGenerator
import pandas as pd



path  = r"/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/MSR-Cambridge/Part2/" 

names = ['TimeStamp','Host_Name','DiskNumber','Operation_Type','ByteOffset','IOSize','Response_Time','DiskNum']


#['TimeStamp','Response','IOType','LUN','ByteOffset','Size']


all_files = glob.glob(os.path.join(path, "src1_1.csv"))
print(all_files)




['/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/MSR-Cambridge/Part2/src1_1.csv']


In [5]:
df = pd.read_csv(all_files[0],engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False) 
names = ['TimeStamp','Host_Name','DiskNumber','Operation_Type','ByteOffset','IOSize','Response_Time','DiskNum']
df.columns = names
#Sorting df by TimeStamp

df = df.sort_values(by=['TimeStamp'])
df.reset_index(inplace=True, drop=True)

print(len(df))

# df['ByteOffset_Delta'] = df['ByteOffset'] - df['ByteOffset'].shift(-1)
# df = df.drop(df.index[-1])

# total_classes = len(Counter(df['ByteOffset_Delta']))
# x = Counter(df['ByteOffset_Delta'])
# vals = {}
# vals =  x.most_common(1000)
# bo_list = []
# coverage = 0
# for key,value in vals:
#     bo_list.append(key)
#     coverage = coverage + value
# print(len(bo_list))
# print("Percentage Coverage Delta")
# print((coverage/len(df))*100) 


# total_classes = len(Counter(df['ByteOffset']))
# x = Counter(df['ByteOffset'])
# vals = {}
# vals =  x.most_common(1000)
# bo_list = []
# coverage = 0
# for key,value in vals:
#     bo_list.append(key)
#     coverage = coverage + value
    
# print(len(bo_list))
# print("Percentage Coverage Offset")
# print((coverage/len(df))*100)
    
 



45746222


In [6]:
# Transform Operation_type, Host_name and remove Disk_Num
# Make Class of Byte Offset delta 
# Normalize the columns (Except ByteOffset Delta)
# Make the prediction model with 1000 classes
import operator
     
# df['ByteOffset_Delta'] = df['ByteOffset_Delta'].fillna(0)
address_map = dict(Counter(df['ByteOffset']))
sorted_address_map = sorted(dict(address_map).items(), key=operator.itemgetter(1))


In [7]:
sorted_address_map[len(sorted_address_map)-2]

(3154132992, 52993)

In [8]:
address_remap = {}

count = 0
while(count<len(sorted_address_map)):
    if(count < 1000):
        address_remap[count] = sorted_address_map[len(sorted_address_map)-count - 1][0]
    else:
        address_remap[1001] = sorted_address_map[len(sorted_address_map)-count - 1][0]
    count = count + 1

In [9]:
rev_address_remap = {y:x for x,y in address_remap.items()}

In [10]:
df.head(5)

,TimeStamp,Host_Name,DiskNumber,Operation_Type,ByteOffset,IOSize,Response_Time,DiskNum
0,128166372013534666,src1,NaN,Write,3180044288,65536,3899,1
1,128166372013535022,src1,NaN,Write,3337596928,65536,3543,1
2,128166372013536418,src1,NaN,Write,102050844672,4096,2148,1
3,128166372013537059,src1,NaN,Write,3180109824,8192,1507,1
4,128166372021660589,src1,NaN,Write,3180118016,28672,2925,1


In [11]:
count = 0
label_list = []
rogue_delta = []
label =0.0
while (count < len(df)):
    x = df['ByteOffset'].iloc[count]
    if x in rev_address_remap:
        label = rev_address_remap[x]
        label_list.append(label)
    else:
        label_list.append(1001)
        rogue_delta.append(x)
    count = count+1

df['ByteOffset_class']  = label_list
print(len(Counter(df['ByteOffset_class'])))
print(len(rogue_delta))

1001
45390748


In [12]:
Counter(label_list)[6]

2368

In [13]:
data_train = df['ByteOffset_class'].values.tolist()
data_test = df['ByteOffset_class'].values.tolist()

In [14]:
# Split to train, validate and test

# Finding the value 75th percentile of TimeStamp

import math


training_pt_1 = math.floor((len(df)*0.75)) 


data_train = df['ByteOffset_class'][:training_pt_1]
data_test = df['ByteOffset_class'][training_pt_1+1:]

print(len(data_train))
print(len(data_test))

34309666
11436555


In [15]:
data_path  = r'/soe/cchakrab/test_output/output_csv/mapping/p1'
path_train  = r"/soe/cchakrab/test_output/output_csv/mapping/p1/train_run1_transfer.txt"

with open(path_train, 'w') as f:
    for item in data_train:
        f.write("%s " % item)
        


In [16]:
path_test  = r"/soe/cchakrab/test_output/output_csv/mapping/p1/test_run1_transfer.txt"
with open(path_test, 'w') as f:
    for item in data_test:
        f.write("%s " % item)

In [17]:
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
import numpy as np
import argparse



def read_words(filename):
    with open(filename, 'r') as f:
        return f.read().split()

def build_vocab(filename):
    data = read_words(filename)

    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))

    return word_to_id


def file_to_word_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]


def load_data():
    # get the data paths
    train_path = os.path.join(data_path, "train_run1_transfer.txt")
    test_path = os.path.join(data_path, "test_run1_transfer.txt")

    # build the complete vocabulary, then convert text data to list of integers
    word_to_id = build_vocab(train_path)
    train_data = file_to_word_ids(train_path, word_to_id)
    test_data = file_to_word_ids(test_path, word_to_id)
    vocabulary = len(word_to_id)
    reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

    print(train_data[:5])
    #print(word_to_id)
    print(vocabulary)
    print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
    return train_data, test_data, vocabulary, reversed_dictionary


In [18]:
train_data, test_data, vocabulary, reversed_dictionary = load_data()

[0, 0, 0, 0, 0]
997
1001 1001 1001 1001 1001 0 3 1001 1001 1001


In [20]:
class KerasBatchGenerator(object):

    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset
        # back to zero
        self.current_idx = 0
        # skip_step is the number of words which will be skipped before the next
        # batch is skimmed from the data set
        self.skip_step = skip_step

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data):
                    # reset the index back to the start of the data set
                    self.current_idx = 0
                x[i, :] = self.data[self.current_idx:self.current_idx + self.num_steps]
                temp_y = self.data[self.current_idx + 1:self.current_idx + self.num_steps + 1]
                # convert all of temp_y into a one hot representation
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                self.current_idx += self.skip_step
            yield x, y


In [21]:
from keras.callbacks import EarlyStopping
look_back = 64
num_steps = look_back
batch_size = 64

train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocabulary, skip_step=num_steps) 
#valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocabulary, skip_step=num_steps)
test_data_generator = KerasBatchGenerator(test_data, num_steps, batch_size, vocabulary, skip_step=num_steps)

hidden_size = 1500
use_dropout=True 

model = Sequential()
model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(vocabulary)))
model.add(Activation('softmax'))


optimizer = Adam()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy',keras.metrics.Precision(), keras.metrics.Recall()])

print(model.summary())
monitor = EarlyStopping(monitor='val_categorical_accuracy', min_delta=1e-3, patience=7, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath=data_path + '/model-trained.hdf5', verbose=1)
num_epochs = 5000

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 64, 1500)          1495500   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64, 1500)          18006000  
_________________________________________________________________
lstm_2 (LSTM)                (None, 64, 1500)          18006000  
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 1500)          0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 64, 997)           1496497   
_________________________________________________________________
activation_1 (Activation)    (None, 64, 997)           0         
Total params: 39,003,997
Trainable params: 39,003,997
Non-trainable params: 0
__________________________________________

In [22]:
model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,
                        validation_data=test_data_generator.generate(),
                        validation_steps=len(train_data)//(batch_size*num_steps), callbacks=[checkpointer,monitor])



/soe/hlitz/miniconda3/envs/py3_tf_gpu/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5000
8376/8376 [==============================] - 3883s 464ms/step - loss: 0.0880 - categorical_accuracy: 0.9918 - precision_1: 0.9922 - recall_1: 0.9913 - val_loss: 0.0621 - val_categorical_accuracy: 0.9927 - val_precision_1: 0.9927 - val_recall_1: 0.9927

Epoch 00001: saving model to /soe/cchakrab/test_output/output_csv/mapping/p1/model-trained.hdf5
Epoch 2/5000
8376/8376 [==============================] - 3874s 463ms/step - loss: 0.0674 - categorical_accuracy: 0.9922 - precision_1: 0.9933 - recall_1: 0.9915 - val_loss: 0.0606 - val_categorical_accuracy: 0.9928 - val_precision_1: 0.9937 - val_recall_1: 0.9920

Epoch 00002: saving model to /soe/cchakrab/test_output/output_csv/mapping/p1/model-trained.hdf5
Epoch 3/5000
5444/8376 [==================>...........] - ETA: 16:02 - loss: 0.0657 - categorical_accuracy: 0.9924 - precision_1: 0.9936 - recall_1: 0.9916

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



8376/8376 [==============================] - 3903s 466ms/step - loss: 0.0680 - categorical_accuracy: 0.9922 - precision_1: 0.9934 - recall_1: 0.9913 - val_loss: 0.0623 - val_categorical_accuracy: 0.9928 - val_precision_1: 0.9934 - val_recall_1: 0.9924

Epoch 00003: saving model to /soe/cchakrab/test_output/output_csv/mapping/p1/model-trained.hdf5
Epoch 4/5000
 152/8376 [..............................] - ETA: 45:09 - loss: 0.1802 - categorical_accuracy: 0.9788 - precision_1: 0.9821 - recall_1: 0.9764

KeyboardInterrupt: 

In [23]:
print("Model trained!")

Model trained!


In [24]:
import numpy as np
import pandas as pd
import glob
import csv
import sys
import os
import math
import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
from sklearn import metrics
from keras.layers.core import Dense, Activation
from keras.callbacks import ModelCheckpoint
from numpy import insert
from sklearn import preprocessing
from collections import Counter
from numpy import array
from numpy import hstack
from keras.preprocessing.sequence import TimeseriesGenerator
import pandas as pd



path  = r"/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/MSR-Cambridge/Part1/" 

names = ['TimeStamp','Host_Name','DiskNumber','Operation_Type','ByteOffset','IOSize','Response_Time','DiskNum']


#['TimeStamp','Response','IOType','LUN','ByteOffset','Size']


all_files = glob.glob(os.path.join(path, "mds_0.csv"))
print(all_files)




['/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/MSR-Cambridge/Part1/mds_0.csv']


In [25]:
df = pd.read_csv(all_files[0],engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False) 
names = ['TimeStamp','Host_Name','DiskNumber','Operation_Type','ByteOffset','IOSize','Response_Time','DiskNum']

df.columns = names
print (len(df))
# df['ByteOffset_Delta'] = df['ByteOffset'] - df['ByteOffset'].shift(-1)
# df = df.drop(df.index[-1])

# total_classes = len(Counter(df['ByteOffset_Delta']))
# x = Counter(df['ByteOffset_Delta'])
# vals = {}
# vals =  x.most_common(1000)
# bo_list = []
# coverage = 0
# for key,value in vals:
#     bo_list.append(key)
#     coverage = coverage + value
# print(len(bo_list))
# print("Percentage Coverage Delta")
# print((coverage/len(df))*100) 



# total_classes = len(Counter(df['ByteOffset']))
# x = Counter(df['ByteOffset'])
# vals = {}
# vals =  x.most_common(1000)
# bo_list = []
# coverage = 0
# for key,value in vals:
#     bo_list.append(key)
#     coverage = coverage + value
    
# print(len(bo_list))
# print("Percentage Coverage Offset")
# print((coverage/len(df))*100)
    
 



1211034


In [26]:
df.tail(5)

,TimeStamp,Host_Name,DiskNumber,Operation_Type,ByteOffset,IOSize,Response_Time,DiskNum
1211029,128172420027898531,mds,NaN,Write,3154137088,4096,46799,0
1211030,128172420027902531,mds,NaN,Write,3161395200,4096,42800,0
1211031,128172420044631296,mds,NaN,Write,1214406656,4096,345394,0
1211032,128172420044657931,mds,NaN,Write,3233218560,4096,318759,0
1211033,128172420044911251,mds,NaN,Write,3221970944,4096,65438,0


In [27]:
# Transform Operation_type, Host_name and remove Disk_Num
# Make Class of Byte Offset delta 
# Normalize the columns (Except ByteOffset Delta)
# Make the prediction model with 1000 classes

     

address_map = dict(Counter(df['ByteOffset']))
sorted_address_map = sorted(dict(address_map).items(), key=operator.itemgetter(1))


In [47]:
address_remap = {}

count = 0
while(count<len(sorted_address_map)):
    if(count < 995):
        address_remap[count] = sorted_address_map[len(sorted_address_map)-count - 1][0]
    else:
        address_remap[995] = sorted_address_map[len(sorted_address_map)-count - 1][0]
    count = count + 1

rev_address_remap = {y:x for x,y in address_remap.items()}

count = 0
label_list = []
rogue_delta = []
label =0.0
while (count < len(df)):
    x = df['ByteOffset'].iloc[count]
    if x in rev_address_remap:
        label = rev_address_remap[x]
        label_list.append(label)
    else:
        label_list.append(1001)
        rogue_delta.append(x)
    count = count+1

df['ByteOffset_class']  = label_list
print(len(Counter(df['ByteOffset_class'])))
print(len(rogue_delta))



997
279551


In [48]:
mapping_test = df['ByteOffset_class'].values.tolist()

path_train  = r"/soe/cchakrab/test_output/output_csv/mapping/p1/mapping_run2_transfer.txt"

with open(path_train, 'w') as f:
    for item in mapping_test:
        f.write("%s " % item)
        



In [49]:
mapping_test = df['ByteOffset_class'].values.tolist()

path_train  = r"/soe/cchakrab/test_output/output_csv/mapping/p1/mapping_run2_transfer.txt"

with open(path_train, 'w') as f:
    for item in mapping_test:
        f.write("%s " % item)
        



In [50]:
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import argparse

data_path = r"/soe/cchakrab/test_output/output_csv/mapping/p1/"

def read_words(filename):
    with open(filename, 'r') as f:
        return f.read().split()

def build_vocab(filename):
    data = read_words(filename)

    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))

    return word_to_id


def file_to_word_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]


def load_data():
    # get the data paths
    train_path = os.path.join(data_path, "mapping_run2_transfer.txt")
    #test_path = os.path.join(data_path, "test_run2_transfer.txt")

    # build the complete vocabulary, then convert text data to list of integers
    word_to_id = build_vocab(train_path)
    train_data = file_to_word_ids(train_path, word_to_id)
    #test_data = file_to_word_ids(test_path, word_to_id)
    vocabulary = len(word_to_id)
    reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

    print(train_data[:5])
    #print(word_to_id)
    print(vocabulary)
    print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
    return train_data, vocabulary, reversed_dictionary


In [51]:
mapping_data, vocabulary, reversed_dictionary = load_data()

[0, 1, 0, 5, 429]
997
1001 0 1001 4 428 427 624 1 623 1001


In [52]:
class KerasBatchGenerator(object):

    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset
        # back to zero
        self.current_idx = 0
        # skip_step is the number of words which will be skipped before the next
        # batch is skimmed from the data set
        self.skip_step = skip_step

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data):
                    # reset the index back to the start of the data set
                    self.current_idx = 0
                x[i, :] = self.data[self.current_idx:self.current_idx + self.num_steps]
                temp_y = self.data[self.current_idx + 1:self.current_idx + self.num_steps + 1]
                # convert all of temp_y into a one hot representation
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                self.current_idx += self.skip_step
            yield x, y


In [53]:
mapping_data_generator = KerasBatchGenerator(mapping_data, num_steps, batch_size, vocabulary, skip_step=num_steps)

In [54]:
import time
start_time = time.time()

samples = (len(mapping_data)/batch_size)
score = model.evaluate_generator(mapping_data_generator.generate(), samples)
print("Loss: ", score[0], "Accuracy: ", score[1])
    
print("--- %s seconds ---" % (time.time() - start_time))


Loss:  7.149606704711914 Accuracy:  0.23075346648693085
--- 2591.0418541431427 seconds ---


In [55]:
print("Done...!")

Done...!
